In [ ]:
from src import utils
imgs, labels = utils.load_imgs('../Wild Cats Dataset', ['CHEETAH' ,'OCELOT', 'SNOW LEOPARD', 'CARACAL', 'LIONS', 'PUMA', 'TIGER'])
x, y = utils.make_dataset(imgs, labels, utils.label_to_idx, (224,224))
print('x shape: {}, y shape:{}'.format(x.shape, y.shape))
utils.plot_sample(imgs, labels, 5, 5, resize=(224, 224))

Loaded 240 images of class CHEETAH
Loaded 238 images of class OCELOT
Loaded 236 images of class SNOW LEOPARD
Loaded 190 images of class CARACAL
Loaded 233 images of class LIONS
Loaded 241 images of class PUMA
Loaded 242 images of class TIGER
Loaded 1620 images total.
x shape: (1620, 224, 224, 3), y shape:(1620, 7)


In [ ]:
################### Task 1 ###################
# Solution go here
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42)
print('x_train shape: {}, y_train shape:{}'.format(x_train.shape, y_train.shape))
print('x_test shape: {}, y_test shape:{}'.format(x_test.shape, y_test.shape))


In [ ]:
################### Task 1 Bonus ###################
# Solution go here






In [ ]:
################### Task 2 ###################
# Solution go here





In [ ]:
################### Task 2 Bonus ###################
# Solution go here




